# Image Forgery Detection using Error Level Analysis (ELA)
In this section, we implement a method to detect tampered or forged images using Error Level Analysis (ELA). ELA helps to visualize areas in an image that have been manipulated by comparing the original image with a compressed version of it.

**What this code does**:
Install and import the necessary Python libraries for image processing and plotting.

Define a function perform_ela() to:

Resave the image with JPEG compression.

Compute the difference between the original and resaved image.

Enhance this difference to highlight inconsistencies.

Define a helper function show_ela_comparison() to visualize both the original and ELA image side by side.

Upload a .zip file containing images.

Extract images, process each one with ELA, and save the ELA results into a separate folder with proper labels:

Images starting with sp are considered forged (label 1).

Images starting with au are considered authentic (label 0).

This pre-processing step helps us build a labeled dataset of ELA images for further analysis or training in machine learning models.

In [ ]:
# ===============================
# Image Forgery Detection with ELA
# ===============================

# 1. Install and import required libraries
!pip install pillow matplotlib

import os
import numpy as np
from PIL import Image, ImageChops, ImageEnhance, UnidentifiedImageError
import matplotlib.pyplot as plt
import zipfile # Import zipfile module

# 2. ELA Function
def perform_ela(image_path, resave_quality=90):
    original = Image.open(image_path).convert('RGB')

    # Resave the image at a lower quality
    resaved_path = 'temp_resaved.jpg'
    original.save(resaved_path, 'JPEG', quality=resave_quality)
    resaved = Image.open(resaved_path)

    # Calculate the difference
    diff = ImageChops.difference(original, resaved)

    # Enhance the difference
    extrema = diff.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff
    ela_image = ImageEnhance.Brightness(diff).enhance(scale)

    # Close images to free up memory
    original.close()
    resaved.close()
    diff.close()

    # Remove temporary file
    os.remove(resaved_path)

    return ela_image

# 3. Display original and ELA image
def show_ela_comparison(image_path):
    try:
        ela_img = perform_ela(image_path)

        # Show images
        fig, axs = plt.subplots(1, 2, figsize=(12, 5))
        axs[0].imshow(Image.open(image_path))
        axs[0].set_title("Original Image")
        axs[0].axis('off')

        axs[1].imshow(ela_img)
        axs[1].set_title("ELA Image")
        axs[1].axis('off')
        plt.show()

        # Close ELA image after showing
        ela_img.close()

    except UnidentifiedImageError:
        print(f"Could not process file {image_path} as an image.")
    except Exception as e:
        print(f"An error occurred processing {image_path}: {e}")

# 4. Upload and test
from google.colab import files
uploaded = files.upload()

# Get the name of the uploaded file (assuming it's a single zip file)
uploaded_file_name = list(uploaded.keys())[0]

# Define a directory to extract the zip file into
extract_dir = './uploaded_images'
os.makedirs(extract_dir, exist_ok=True)

# Extract the uploaded zip file
try:
    with zipfile.ZipFile(uploaded_file_name, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Extracted {uploaded_file_name} to {extract_dir}")

    # Prepare output folder for ELA images
    ela_output_dir = './ela_images'
    os.makedirs(ela_output_dir, exist_ok=True)

    # Process each file in the extracted directory
    print("Processing extracted files:")
    for root, _, files in os.walk(extract_dir):
        for file in files:
            file_path = os.path.join(root, file)
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                print(f"Processing {file_path}")
                show_ela_comparison(file_path)

                try:
                    ela_img = perform_ela(file_path)

                    # file name
                    filename_lower = file.lower()
                    if filename_lower.startswith('sp'):
                        label = 1  # fake
                    elif filename_lower.startswith('au'):
                        label = 0  # authentic
                    else:
                        label = -1  # skip

                    if label != -1:
                        save_name = f"{label}_{file}"
                        save_path = os.path.join(ela_output_dir, save_name)
                        ela_img.save(save_path)
                    else:
                        print(f"⚠️ DOSE NOT BY 'SP' OR 'Au': {file_path}")

                    ela_img.close()
                except Exception as e:
                    print(f"⚠️ WRONG {file_path}: {e}")

            else:
                print(f"Skipping non-image file: {file_path}")

except zipfile.BadZipFile:
    print(f"Error: The uploaded file '{uploaded_file_name}' is not a valid zip file.")
except Exception as e:
    print(f"An error occurred during file extraction or processing: {e}")


# **Training a Deep Learning Model for Image Forgery Detection**
In this section, we build and train a deep learning model based on MobileNetV2, a lightweight convolutional neural network pre-trained on ImageNet. The goal is to classify ELA images as either authentic (0) or forged (1).

**Steps performed**:
**Load and preprocess data**:

Read images from the ela_images directory.

Resize them to 128x128 and normalize pixel values.

Labels are extracted from the filename (0 = authentic, 1 = fake).

**Split data**:

Use train_test_split to divide data into training (70%) and testing (30%).

**Handle class imbalance**:

Compute class_weight to balance the dataset during training.

**Data augmentation**:

Apply random transformations like rotation and flipping to improve generalization using ImageDataGenerator.

**Build model**:

Use MobileNetV2 as the base feature extractor (frozen first, then fine-tuned).

Add pooling, dropout, and a dense output layer with sigmoid activation for binary classification.

**Train in two phases**:

Initial training with the base model frozen (10 epochs).

Fine-tuning with some layers unfrozen (20 epochs).

**Evaluate the model**:

Report accuracy and loss on the test set.

Save the best model to best_mobilenetv2_model.h5.

**Final Results**:
Total Images Used: 1845

Test Accuracy: ~71.7%

Test Loss: ~0.54

**Model Parameters**: ~4.67M

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import random

seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

# LOAD DATA
ela_dir = './ela_images'
image_size = (128, 128)
X, y = [], []

for file in os.listdir(ela_dir):
    if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        try:
            label = int(file.split('_')[0])
        except:
            continue
        img_path = os.path.join(ela_dir, file)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, image_size)
            X.append(img)
            y.append(label)

print(f"✅ Prepared {len(X)} images for the dataset.")

if len(X) > 0:
    X = np.array(X) / 255.0
    y = np.array(y)


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)
    print(f"📊 Training data size: {X_train.shape}, Test data size: {X_test.shape}")


    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
    class_weight_dict = dict(enumerate(class_weights))
    print(f"🧮 Class weights: {class_weight_dict}")


    datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True
    )
    datagen.fit(X_train)

    #  MobileNetV2
    base_model = MobileNetV2(input_shape=(128,128,3), include_top=False, weights='imagenet')
    base_model.trainable = False

    inputs = Input(shape=(128,128,3))
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    outputs = Dense(1, activation='sigmoid')(x)
    model = Model(inputs, outputs)

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    history = model.fit(
        datagen.flow(X_train, y_train, batch_size=32),
        epochs=10,
        validation_data=(X_test, y_test),
        class_weight=class_weight_dict,
        callbacks=[early_stop]
    )

    base_model.trainable = True
    for layer in base_model.layers[:-20]:
        layer.trainable = False

    model.compile(optimizer=Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy'])

    fine_tune_history = model.fit(
        datagen.flow(X_train, y_train, batch_size=32),
        epochs=20,
        validation_data=(X_test, y_test),
        class_weight=class_weight_dict,
        callbacks=[early_stop]
    )

    model.save('best_mobilenetv2_model.h5')

    test_loss, test_acc = model.evaluate(X_test, y_test)
    print(f"Test Accuracy: {test_acc:.2f}, Test Loss: {test_loss:.2f}")

    model.summary()

else:
    print("❌ No images found.")



# 📊 Model Training Performance
We visualize the training and validation performance over the training epochs.

Accuracy Plot shows how the model's accuracy evolved on both training and validation sets.

Loss Plot illustrates the error during training and helps in identifying overfitting or underfitting.

These visualizations help in understanding the model's learning behavior and making decisions about further tuning.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()


In [ ]:
# Install required packages
!pip install gradio Pillow

import numpy as np
import tensorflow as tf
from PIL import Image, ImageChops, ImageEnhance
import gradio as gr
import io
import os

# Load your trained model
model = tf.keras.models.load_model('best_mobilenetv2_model.h5')

def perform_ela(img, resave_quality=90):
    """Perform Error Level Analysis on an image"""
    if isinstance(img, np.ndarray):
        # Convert numpy array to PIL Image
        original = Image.fromarray(img.astype('uint8'))
    else:
        # Convert Gradio image to PIL Image
        original = Image.fromarray(img)

    # Save to buffer with compression
    buffer = io.BytesIO()
    original.save(buffer, 'JPEG', quality=resave_quality)
    buffer.seek(0)
    resaved = Image.open(buffer)

    # Calculate difference
    diff = ImageChops.difference(original, resaved)

    # Enhance difference
    extrema = diff.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff
    ela_image = ImageEnhance.Brightness(diff).enhance(scale)

    return ela_image

def preprocess_image(ela_img):
    """Preprocess ELA image for model input"""
    ela_img = ela_img.resize((128, 128))
    if ela_img.mode != 'RGB':
        ela_img = ela_img.convert('RGB')
    ela_array = np.array(ela_img) / 255.0
    return np.expand_dims(ela_array, axis=0)

def predict_image(img):
    """Make prediction on uploaded image"""
    # Generate ELA image
    ela_img = perform_ela(img)

    # Preprocess for model
    processed_img = preprocess_image(ela_img)

    # Make prediction
    prediction = model.predict(processed_img)[0][0]
    forged_prob = float(prediction)
    authentic_prob = 1 - forged_prob

    # Create label dictionary
    labels = {
        "Authentic": authentic_prob,
        "Forged": forged_prob
    }

    return ela_img, labels

# Create Gradio interface
title = "Image Forgery Detection using ELA"
description = (
    "Upload an image to check if it's authentic or forged. "
    "The system uses Error Level Analysis (ELA) and a trained MobileNetV2 model."
    "<br><br><b>How it works:</b>"
    "<br>1. ELA highlights compression artifacts that reveal image tampering"
    "<br>2. The AI model analyzes the ELA image"
    "<br>3. Predictions show authenticity probability"
)

iface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(label="Upload Image"),
    outputs=[
        gr.Image(label="ELA Processed Image"),
        gr.Label(label="Prediction", num_top_classes=2)
    ],
    title=title,
    description=description,
    allow_flagging="never",
    examples=[
        ["sample_authentic.jpg"],  # Replace with your sample images
        ["sample_forged.jpg"]      # Replace with your sample images
    ]
)

iface.launch(share=True)